In [ ]:
!pip install transformers wikipedia newspaper3k GoogleNews pyvis

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.7 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=242b82511f5215dc53cb254e79e40c3bbcefa719876dc05765a98d716aa4d294
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76ed

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import math
import torch
import wikipedia
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
import IPython
from pyvis.network import Network
import json
import spacy

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
# # Load model directly

# tokenizer1 = AutoTokenizer.from_pretrained("dhtocks/Named-Entity-Recognition")
# model1 = AutoModelForTokenClassification.from_pretrained("dhtocks/Named-Entity-Recognition")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
def extract_relations_from_model_output(text):
    relation, subject, object_ = '', '', ''
    current = 'x'

    # Remove specific tokens from the input text
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")

    # Initialize a list to store extracted relations
    relations = []

    # Iterate through tokens in the modified text
    for token in text_replaced.split():
        if token == "<triplet>":
            # Start processing a new triplet
            current = 't'

            # If a complete relation exists, add it to the list
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            # Start processing the subject component
            current = 's'

            # If a complete relation exists, add it to the list
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            # Start processing the object component
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token

    # Check if there is a complete relation at the end and add it to the list
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })

    # Return the extracted relations
    return relations


In [ ]:
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [ ]:
class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [ ]:
# Define a function to remove empty lines and lines containing only '----------' from an array of text
def remove_empty_lines(array):
    filtered_lines = []

    for line in array:
        line = line.strip()  # Remove leading/trailing whitespaces
        if line != '' and line != '----------':
            filtered_lines.append(line)

    return filtered_lines

# Open a text file for reading /content/test.txt
with open("/content/1-s2.0-S0002937824000127-main.pdf.txt") as f:
    txt = f.readlines()  # Read the lines from the file

    # Remove empty lines and lines containing only '----------'
    filtered_txt = remove_empty_lines(txt)

    # Combine the filtered lines into a single string
    result = ' '.join(filtered_txt)

    print(result)
#/content/1-s2.0-S0002937824000127-main.pdf.txt
    kb = from_text_to_kb(result, verbose=True)  # Convert text to a knowledge base

    # Print the knowledge base (kb) if available
    kb.print()


Token indices sequence length is longer than the specified maximum sequence length for this model (16496 > 1024). Running this sequence through the model will result in indexing errors


Page 1 Text: Clinical Opinion Gestational diabetes in twin pregnancies—a pathology requiring treatment or a benign physiological adaptation? Nir Melamed, MD, MSc; Tomer Avnon, MD; Jon Barrett, MD; Nathan Fox, MD; ajog.org Andrei Rebarber, MD; Baiju R. Shah, MD, PhD; Ilana Halperin, MD; Ravi Retnakaran, MD; Howard Berger, MD; John Kingdom, MD; Liran Hiersch, MD There is level-I evidence that screening for and treating gestational diabetes in singleton pregnancies reduce maternal and neonatal morbidity. However, similar data for gestational diabetes in twin pregnancies are currently lacking. Consequently, the current approach for the diagnosis and management of gestational diabetes in twin pregnancies is based on the same diagnostic criteria and glycemic targets used in singleton pregnancies. However, twin pregnancies have unique physiological characteristics, and many of the typical gestational diabetes-related complications are less relevant for twin pregnancies. These differences rais

In [ ]:
from pyvis.network import Network  # Import the necessary library

# Define a function to save a network visualization to an HTML file
def save_network_html(kb, filename="network.html"):
    net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee")

    color_entity = "#00FF00"  # Define the node color for entities
    for e in kb.entities:
        net.add_node(e, shape="circle", color=color_entity)

    for r in kb.relations:
        net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])

    # Configure network layout and appearance
    net.repulsion(
        node_distance=200,
        central_gravity=0.2,
        spring_length=200,
        spring_strength=0.05,
        damping=0.09
    )
    net.set_edge_smooth('dynamic')

    # Save the network to the specified HTML file
    net.save_graph(filename)


In [ ]:
import pickle

# Define a function to save a knowledge base (kb) to a file
def save_kb(kb, filename):
    with open(filename, "wb") as f:
        pickle.dump(kb, f)

# Define a filename for the HTML network visualization
filename = "network_1_google.html"

save_network_html(kb, filename=filename)

new_filename = filename.split(".")[0] + ".p"

# Save the knowledge base (kb) to a pickle file with the new filename
save_kb(kb, filename=new_filename)

IPython.display.HTML(filename=filename)


In [ ]:
kdict = vars(kb)
kdict

{'entities': {'Hypervolemia': {'url': 'https://en.wikipedia.org/wiki/Hypervolemia',
   'summary': 'Hypervolemia, also known as fluid overload, is the medical condition where there is too much fluid in the blood. The opposite condition is hypovolemia, which is too little fluid volume in the blood. Fluid volume excess in the intravascular compartment occurs due to an increase in total body sodium content and a consequent increase in extracellular body water. The mechanism usually stems from compromised regulatory mechanisms for sodium handling as seen in congestive heart failure (CHF), kidney failure, and liver failure. It may also be caused by excessive intake of sodium from foods, intravenous (IV) solutions and blood transfusions, medications, or diagnostic contrast dyes. Treatment typically includes administration of diuretics and limit the intake of water, fluids, sodium, and salt.\n\n'},
  'Circulatory system': {'url': 'https://en.wikipedia.org/wiki/Circulatory_system',
   'summary'

In [ ]:
file = open('/content/message.txt','r')
file.read()

"Entity Typing for Node Names in the Medical Domain:\n\n1. 'Hypervolemia'\n   - Entity Type: Medical Condition\n\n2. 'Circulatory System'\n   - Entity Type: Biological System\n\n3. 'Heart Failure with Preserved Ejection Fraction'\n   - Entity Type: Medical Condition\n\n4. 'Cohort Study'\n   - Entity Type: Research Study Type\n\n5. 'Retrospective'\n   - Entity Type: Research Study Type\n\n6. 'Meta-analysis'\n   - Entity Type: Research Study Type\n\n7. 'Systematic Review'\n   - Entity Type: Research Study Type\n\n8. 'Scientific Evidence'\n   - Entity Type: Concept\n\n9. 'Diastole'\n   - Entity Type: Cardiac Cycle Phase\n\n10. 'Systole'\n    - Entity Type: Cardiac Cycle Phase\n\n11. 'Heart Rate'\n    - Entity Type: Physiological Measurement\n\n12. 'Healthcare Cost and Utilization Project'\n    - Entity Type: Healthcare Database/Project\n\n13. 'United States'\n    - Entity Type: Geographic Location\n\n14. 'Preterm Birth'\n    - Entity Type: Medical Condition\n\n15. 'Hypertension'\n    - En

In [ ]:
data = [
    {'head': 'Postpartum period', 'type': 'follows', 'tail': 'Postpartum period', 'meta': {'spans': [[254, 382], [2413, 2541]]}},
    {'head': 'Anatomy', 'type': 'has part', 'tail': 'Physiology', 'meta': {'spans': [[254, 382]]}},
    {'head': 'Physiology', 'type': 'part of', 'tail': 'Anatomy', 'meta': {'spans': [[254, 382]]}},
    {'head': 'Anatomy', 'type': 'part of', 'tail': 'Physiology', 'meta': {'spans': [[254, 382]]}},
    {'head': 'Physiology', 'type': 'has part', 'tail': 'Anatomy', 'meta': {'spans': [[254, 382]]}},
  {'head': 'Physical Activity Guidelines for Americans', 'type': 'country', 'tail': 'United States', 'meta': {'spans': [[762, 890]]}},
  {'head': 'Cardiology', 'type': 'has part', 'tail': 'Pulmonology', 'meta': {'spans': [[1016, 1144]]}},
  {'head': 'Water aerobics', 'type': 'subclass of', 'tail': 'Aerobic exercise', 'meta': {'spans': [[1397, 1525]]}},
  {'head': 'Pre-eclampsia', 'type': 'subclass of', 'tail': 'Gestational hypertension', 'meta': {'spans': [[1397, 1525]]}},
  {'head': 'Pregnancy', 'type': 'followed by', 'tail': 'Pregnancy', 'meta': {'spans': [[1651, 1779]]}},
  {'head': 'Pregnancy', 'type': 'follows', 'tail': 'Pregnancy', 'meta': {'spans': [[1651, 1779]]}},
  {'head': 'Pregnancy', 'type': 'part of', 'tail': 'Pregnancy', 'meta': {'spans': [[1651, 1779], [1905, 2033], [3937, 4065]]}},
  {'head': 'Neural tube defect', 'type': 'subclass of', 'tail': 'Neural tube defect', 'meta': {'spans': [[1778, 1906]]}},
  {'head': 'Birth weight', 'type': 'facet of', 'tail': 'Birth weight', 'meta': {'spans': [[1778, 1906]]}},
  {'head': 'Pregnancy', 'type': 'has part', 'tail': 'Pregnancy', 'meta': {'spans': [[1905, 2033]]}},
  {'head': 'Meta-analysis', 'type': 'facet of', 'tail': 'Systematic review', 'meta': {'spans': [[2921, 3049], [6604, 6732], [6985, 7113]]}},
  {'head': 'Pregnancy', 'type': 'has effect', 'tail': 'Pregnancy', 'meta': {'spans': [[3175, 3303]]}},
  {'head': 'Pregnancy', 'type': 'has cause', 'tail': 'Pregnancy', 'meta': {'spans': [[3175, 3303]]}},
  {'head': 'Jogging', 'type': 'use', 'tail': 'Aerobics', 'meta': {'spans': [[3429, 3557]]}},
  {'head': 'Jogging', 'type': 'subclass of', 'tail': 'Aerobics', 'meta': {'spans': [[3429, 3557]]}},
  {'head': 'Air conditioning', 'type': 'opposite of', 'tail': 'Heat', 'meta': {'spans': [[3556, 3684]]}},
  {'head': 'Air conditioning', 'type': 'facet of', 'tail': 'Heat', 'meta': {'spans': [[3556, 3684]]}},
  {'head': 'Randomized controlled trial', 'type': 'subclass of', 'tail': 'Trial', 'meta': {'spans': [[3937, 4065]]}},
  {'head': 'Randomized controlled trial', 'type': 'subclass of', 'tail': 'Randomized controlled trial', 'meta': {'spans': [[3937, 4065], [5461, 5589], [5588, 5716], [6350, 6478]]}},
  {'head': 'Perineum', 'type': 'part of', 'tail': 'Childbirth', 'meta': {'spans': [[4064, 4192]]}},
  {'head': 'Perineum', 'type': 'has effect', 'tail': 'Childbirth', 'meta': {'spans': [[4064, 4192]]}},
  {'head': 'Childbirth', 'type': 'has cause', 'tail': 'Perineum', 'meta': {'spans': [[4064, 4192]]}},
  {'head': 'Anti-gravity', 'type': 'opposite of', 'tail': 'Gravity', 'meta': {'spans': [[4318, 4446]]}},
  {'head': 'Anti-gravity', 'type': 'subclass of', 'tail': 'Gravity', 'meta': {'spans': [[4318, 4446]]}},
  {'head': 'Gravity', 'type': 'opposite of', 'tail': 'Anti-gravity', 'meta': {'spans': [[4318, 4446]]}},
  {'head': 'Obesity', 'type': 'has cause', 'tail': 'Overweight', 'meta': {'spans': [[4699, 4827]]}},
  {'head': 'Overweight', 'type': 'has effect', 'tail': 'Obesity', 'meta': {'spans': [[4699, 4827]]}},
  {'head': 'Obesity', 'type': 'subclass of', 'tail': 'Overweight', 'meta': {'spans': [[4699, 4827]]}},
  {'head': 'World Health Organization', 'type': 'headquarters location', 'tail': 'Geneva', 'meta': {'spans': [[5207, 5335]]}},
  {'head': 'Randomized controlled trial', 'type': 'facet of', 'tail': 'Preterm birth', 'meta': {'spans': [[5588, 5716]]}},
  {'head': 'Pregnancy', 'type': 'followed by', 'tail': 'Postpartum period', 'meta': {'spans': [[5715, 5843], [7874, 8002]]}},
  {'head': 'Postpartum period', 'type': 'follows', 'tail': 'Pregnancy', 'meta': {'spans': [[5715, 5843], [6731, 6859], [7874, 8002], [8128, 8256]]}},
  {'head': 'Systematic review', 'type': 'subclass of', 'tail': 'Systematic review', 'meta': {'spans': [[5842, 5970]]}},
  {'head': 'British Journal of Sports Medicine', 'type': 'instance of', 'tail': 'Systematic review', 'meta': {'spans': [[5842, 5970]]}},
  {'head': 'List of Hindawi academic journals', 'type': 'instance of', 'tail': 'Systematic review', 'meta': {'spans': [[5842, 5970]]}},
  {'head': 'Pregnancy', 'type': 'subclass of', 'tail': 'Exercise', 'meta': {'spans': [[6223, 6351]]}},
  {'head': 'Systematic review', 'type': 'has part', 'tail': 'Meta-analysis', 'meta': {'spans': [[6604, 6732], [6985, 7113]]}},
  {'head': 'Meta-analysis', 'type': 'part of', 'tail': 'Systematic review', 'meta': {'spans': [[6604, 6732], [6985, 7113]]}},
  {'head': 'Pregnancy', 'type': 'has effect', 'tail': 'Postpartum period', 'meta': {'spans': [[6731, 6859], [7874, 8002], [8128, 8256]]}},
  {'head': 'Pre-eclampsia', 'type': 'subclass of', 'tail': 'Pre-existing condition', 'meta': {'spans': [[6731, 6859]]}},
  {'head': 'American College of Obstetricians and Gynecologists', 'type': 'located in the administrative territorial entity', 'tail': 'Washington, D.C.', 'meta': {'spans': [[7239, 7367], [8128, 8256]]}},
  {'head': 'American College of Obstetricians and Gynecologists', 'type': 'headquarters location', 'tail': 'Washington, D.C.', 'meta': {'spans': [[7239, 7367]]}},
  {'head': 'Randomized controlled trial', 'type': 'subclass of', 'tail': 'Clinical trial', 'meta': {'spans': [[7366, 7494]]}},
  {'head': 'Clinical trial', 'type': 'subclass of', 'tail': 'Randomized controlled trial', 'meta': {'spans': [[7366, 7494]]}},
  {'head': 'Cochrane Library', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[7620, 7748]]}},
  {'head': 'Cochrane Library', 'type': 'instance of', 'tail': 'Systematic review', 'meta': {'spans': [[7620, 7748]]}},
  {'head': 'Pre-eclampsia', 'type': 'has cause', 'tail': 'Low birth weight', 'meta': {'spans': [[7747, 7875]]}},
  {'head': 'Tehran', 'type': 'country', 'tail': 'Iran', 'meta': {'spans': [[8636, 8764]]}},
  {'head': 'Pregnancy', 'type': 'has part', 'tail': 'Infant', 'meta': {'spans': [[8890, 9018]]}},
  {'head': 'Infant', 'type': 'part of', 'tail': 'Pregnancy', 'meta': {'spans': [[8890, 9018]]}},
  {'head': 'Pathophysiology',
   'type': 'part of',
   'tail': 'Epidemiology',
   'meta': {'spans': [[254, 382], [8763, 8891]]}},
  {'head': 'Epidemiology',
   'type': 'part of',
   'tail': 'Pathophysiology',
   'meta': {'spans': [[254, 382], [8763, 8891]]}},
  {'head': 'Epidemiology',
   'type': 'has part',
   'tail': 'Pathophysiology',
   'meta': {'spans': [[254, 382], [8763, 8891]]}},
  {'head': 'Gestational diabetes',
   'type': 'facet of',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[381, 509]]}},
  {'head': 'Gestational diabetes',
   'type': 'has effect',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[381, 509]]}},
  {'head': 'Sunnybrook Health Sciences Centre',
   'type': 'affiliation',
   'tail': 'University of Toronto',
   'meta': {'spans': [[508, 636], [762, 890]]}},
  {'head': 'Sunnybrook Health Sciences Centre',
   'type': 'located in the administrative territorial entity',
   'tail': 'Ontario',
   'meta': {'spans': [[508, 636]]}},
  {'head': 'University of Toronto',
   'type': 'subsidiary',
   'tail': 'Sunnybrook Health Sciences Centre',
   'meta': {'spans': [[508, 636], [762, 890]]}},
  {'head': 'University of Toronto',
   'type': 'located in the administrative territorial entity',
   'tail': 'Ontario',
   'meta': {'spans': [[508, 636], [889, 1017]]}},
  {'head': 'Medical school',
   'type': 'part of',
   'tail': 'University of Toronto',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Medical school',
   'type': 'part of',
   'tail': 'Tel Aviv University',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Tel Aviv Sourasky Medical Center',
   'type': 'affiliation',
   'tail': 'Tel Aviv University',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Tel Aviv University',
   'type': 'located in the administrative territorial entity',
   'tail': 'Tel Aviv',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Tel Aviv University',
   'type': 'country',
   'tail': 'Israel',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Tel Aviv',
   'type': 'country',
   'tail': 'Israel',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'McMaster University',
   'type': 'located in the administrative territorial entity',
   'tail': 'Hamilton, Ontario',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Hamilton, Ontario',
   'type': 'located in the administrative territorial entity',
   'tail': 'Hamilton, Ontario',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'Icahn School of Medicine at Mount Sinai',
   'type': 'subsidiary',
   'tail': 'Mount Sinai',
   'meta': {'spans': [[635, 763]]}},
  {'head': 'University of Toronto',
   'type': 'part of',
   'tail': 'University of Toronto',
   'meta': {'spans': [[762, 890]]}},
  {'head': 'University of Toronto',
   'type': 'location',
   'tail': 'Toronto',
   'meta': {'spans': [[889, 1017]]}},
  {'head': 'Toronto',
   'type': 'located in the administrative territorial entity',
   'tail': 'Ontario',
   'meta': {'spans': [[889, 1017]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'publisher',
   'tail': 'Elsevier',
   'meta': {'spans': [[1016, 1144]]}},
  {'head': 'Insulin resistance',
   'type': 'instance of',
   'tail': 'Physiology',
   'meta': {'spans': [[1397, 1525]]}},
  {'head': 'Insulin resistance',
   'type': 'subclass of',
   'tail': 'Glycemic',
   'meta': {'spans': [[1397, 1525]]}},
  {'head': 'Ontario',
   'type': 'country',
   'tail': 'Canada',
   'meta': {'spans': [[1651, 1779],
     [3556, 3684],
     [6731, 6859],
     [7620, 7748]]}},
  {'head': 'Canada',
   'type': 'contains administrative territorial entity',
   'tail': 'Ontario',
   'meta': {'spans': [[1651, 1779],
     [3556, 3684],
     [6731, 6859],
     [7620, 7748]]}},
  {'head': 'Insulin resistance',
   'type': 'has effect',
   'tail': 'Type 2 diabetes',
   'meta': {'spans': [[2032, 2160]]}},
  {'head': 'Type 2 diabetes',
   'type': 'has cause',
   'tail': 'Insulin resistance',
   'meta': {'spans': [[2032, 2160], [2921, 3049], [9525, 9653]]}},
  {'head': 'Caesarean section',
   'type': 'has cause',
   'tail': 'Pre-eclampsia',
   'meta': {'spans': [[2286, 2414]]}},
  {'head': 'Caesarean section',
   'type': 'has effect',
   'tail': 'Pre-eclampsia',
   'meta': {'spans': [[2286, 2414]]}},
  {'head': 'Caesarean section',
   'type': 'has cause',
   'tail': 'Preterm birth',
   'meta': {'spans': [[2286, 2414]]}},
  {'head': 'Confidence interval',
   'type': 'subclass of',
   'tail': 'P-value',
   'meta': {'spans': [[2667, 2795]]}},
  {'head': 'Chlorine',
   'type': 'instance of',
   'tail': 'Confidence interval',
   'meta': {'spans': [[2667, 2795]]}},
  {'head': 'Type 2 diabetes',
   'type': 'subclass of',
   'tail': 'Insulin resistance',
   'meta': {'spans': [[2921, 3049], [9525, 9653]]}},
  {'head': 'Diabetes',
   'type': 'has effect',
   'tail': 'Insulin resistance',
   'meta': {'spans': [[3048, 3176]]}},
  {'head': 'Diabetes',
   'type': 'has cause',
   'tail': 'Insulin resistance',
   'meta': {'spans': [[3048, 3176]]}},
  {'head': 'Insulin resistance',
   'type': 'has effect',
   'tail': 'Diabetes',
   'meta': {'spans': [[3048, 3176]]}},
  {'head': 'Apgar score',
   'type': 'instance of',
   'tail': 'Apgar',
   'meta': {'spans': [[3175, 3303]]}},
  {'head': 'Meta-analysis',
   'type': 'subclass of',
   'tail': 'Meta-analysis',
   'meta': {'spans': [[3175, 3303]]}},
  {'head': 'Shortness of breath',
   'type': 'subclass of',
   'tail': 'Shortness of breath',
   'meta': {'spans': [[3175, 3303]]}},
  {'head': 'Ontario',
   'type': 'located in the administrative territorial entity',
   'tail': 'Canada',
   'meta': {'spans': [[3556, 3684], [6731, 6859]]}},
  {'head': 'Type 2 diabetes',
   'type': 'subclass of',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[4191, 4319]]}},
  {'head': 'Infant',
   'type': 'part of',
   'tail': 'Mother',
   'meta': {'spans': [[4318, 4446]]}},
  {'head': 'Mother',
   'type': 'followed by',
   'tail': 'Infant',
   'meta': {'spans': [[4318, 4446]]}},
  {'head': 'Twin',
   'type': 'subclass of',
   'tail': 'Pregnancy',
   'meta': {'spans': [[4953, 5081]]}},
  {'head': 'Twin',
   'type': 'subclass of',
   'tail': 'Birth weight',
   'meta': {'spans': [[4953, 5081]]}},
  {'head': 'Insulin',
   'type': 'subclass of',
   'tail': 'Blood sugar level',
   'meta': {'spans': [[5080, 5208]]}},
  {'head': 'Insulin-like growth factor',
   'type': 'subclass of',
   'tail': 'Growth factor',
   'meta': {'spans': [[5080, 5208]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'main subject',
   'tail': 'Pregnancy',
   'meta': {'spans': [[5588, 5716]]}},
  {'head': 'Glucose tolerance test',
   'type': 'instance of',
   'tail': 'Diagnosis',
   'meta': {'spans': [[6858, 6986]]}},
  {'head': 'Jaundice',
   'type': 'subclass of',
   'tail': 'Hypoglycemia',
   'meta': {'spans': [[7112, 7240]]}},
  {'head': 'Jaundice',
   'type': 'has cause',
   'tail': 'Hypoglycemia',
   'meta': {'spans': [[7112, 7240]]}},
  {'head': 'Hypoglycemia',
   'type': 'has effect',
   'tail': 'Jaundice',
   'meta': {'spans': [[7112, 7240]]}},
  {'head': 'Neonatal intensive care unit',
   'type': 'part of',
   'tail': 'Neonatal intensive care unit',
   'meta': {'spans': [[7366, 7494]]}},
  {'head': 'Neonatal intensive care unit',
   'type': 'has part',
   'tail': 'Neonatal intensive care unit',
   'meta': {'spans': [[7366, 7494]]}},
  {'head': 'Insulin resistance',
   'type': 'facet of',
   'tail': 'Physiology',
   'meta': {'spans': [[7493, 7621]]}},
  {'head': 'Type 2 diabetes',
   'type': 'subclass of',
   'tail': 'Prediabetes',
   'meta': {'spans': [[8001, 8129]]}},
  {'head': 'Type 2 diabetes',
   'type': 'has cause',
   'tail': 'Prediabetes',
   'meta': {'spans': [[8001, 8129]]}},
  {'head': 'Type 2 diabetes',
   'type': 'has effect',
   'tail': 'Prediabetes',
   'meta': {'spans': [[8001, 8129]]}},
  {'head': 'Type 2 diabetes',
   'type': 'subclass of',
   'tail': 'Diabetes',
   'meta': {'spans': [[8255, 8383], [10160, 10288]]}},
  {'head': 'Diabetes',
   'type': 'has effect',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[8255, 8383], [12192, 12320], [16002, 16130]]}},
  {'head': 'Gestational diabetes',
   'type': 'subclass of',
   'tail': 'Diabetes',
   'meta': {'spans': [[8255, 8383], [16002, 16130], [16256, 16384]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'part of',
   'tail': 'PubMed',
   'meta': {'spans': [[8890, 9018]]}},
  {'head': 'Glucose tolerance test',
   'type': 'instance of',
   'tail': 'Glucose tolerance test',
   'meta': {'spans': [[9398, 9526]]}},
  {'head': 'Glucose tolerance test',
   'type': 'subclass of',
   'tail': 'Glucose tolerance test',
   'meta': {'spans': [[9398, 9526]]}},
  {'head': 'Insulin resistance',
   'type': 'has cause',
   'tail': 'Type 2 diabetes',
   'meta': {'spans': [[9525, 9653]]}},
  {'head': 'Type 2 diabetes',
   'type': 'has effect',
   'tail': 'Insulin resistance',
   'meta': {'spans': [[9525, 9653]]}},
  {'head': 'Glucose tolerance test',
   'type': 'instance of',
   'tail': 'Medical diagnosis',
   'meta': {'spans': [[9779, 9907]]}},
  {'head': 'Low birth weight',
   'type': 'facet of',
   'tail': 'Diabetes',
   'meta': {'spans': [[9906, 10034]]}},
  {'head': 'Sunnybrook Health Sciences Centre',
   'type': 'located in the administrative territorial entity',
   'tail': 'Toronto',
   'meta': {'spans': [[10033, 10161]]}},
  {'head': 'Sunnybrook Health Sciences Centre',
   'type': 'country',
   'tail': 'Canada',
   'meta': {'spans': [[10033, 10161]]}},
  {'head': 'Toronto',
   'type': 'country',
   'tail': 'Canada',
   'meta': {'spans': [[10033, 10161]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'publication date',
   'tail': '2024',
   'meta': {'spans': [[10287, 10415], [12700, 12828]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'inception',
   'tail': '1985',
   'meta': {'spans': [[10795, 10923]]}},
  {'head': 'Twin Research and Human Genetics',
   'type': 'publication date',
   'tail': '1991',
   'meta': {'spans': [[11176, 11304]]}},
  {'head': 'Twin Research and Human Genetics',
   'type': 'inception',
   'tail': '1991',
   'meta': {'spans': [[11176, 11304]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'instance of',
   'tail': 'Systematic review',
   'meta': {'spans': [[11430, 11558]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'inception',
   'tail': '1991',
   'meta': {'spans': [[11557, 11685]]}},
  {'head': 'Growth hormone 2',
   'type': 'part of',
   'tail': 'Prenatal development',
   'meta': {'spans': [[11557, 11685]]}},
  {'head': 'Meta-analysis',
   'type': 'facet of',
   'tail': 'Systematic review',
   'meta': {'spans': [[11938, 12066]]}},
  {'head': 'Diabetes and pregnancy',
   'type': 'facet of',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[12192, 12320]]}},
  {'head': 'Diabetes and pregnancy',
   'type': 'main subject',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[12192, 12320]]}},
  {'head': 'Diabetes',
   'type': 'has effect',
   'tail': 'Hypertension',
   'meta': {'spans': [[12446, 12574]]}},
  {'head': 'Hypertension',
   'type': 'has cause',
   'tail': 'Diabetes',
   'meta': {'spans': [[12446, 12574]]}},
  {'head': 'Type 2 diabetes',
   'type': 'subclass of',
   'tail': 'Hypertension',
   'meta': {'spans': [[12573, 12701]]}},
  {'head': 'Type 1 diabetes',
   'type': 'subclass of',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[12573, 12701]]}},
  {'head': 'Gestational diabetes',
   'type': 'subclass of',
   'tail': 'Type 1 diabetes',
   'meta': {'spans': [[12573, 12701]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'instance of',
   'tail': 'Scientific journal',
   'meta': {'spans': [[12700, 12828]]}},
  {'head': 'Twin',
   'type': 'subclass of',
   'tail': 'Birth defect',
   'meta': {'spans': [[14605, 14733]]}},
  {'head': 'Twin',
   'type': 'instance of',
   'tail': 'Birth defect',
   'meta': {'spans': [[14605, 14733]]}},
  {'head': 'Hypertension',
   'type': 'subclass of',
   'tail': 'Blood pressure',
   'meta': {'spans': [[15621, 15749]]}},
  {'head': 'Hypertension',
   'type': 'has effect',
   'tail': 'Blood pressure',
   'meta': {'spans': [[15621, 15749]]}},
  {'head': 'American Journal of Obstetrics and Gynecology',
   'type': 'area',
   'tail': '2',
   'meta': {'spans': [[15748, 15876]]}},
  {'head': 'Journal of Perinatology',
   'type': 'publication date',
   'tail': '2014',
   'meta': {'spans': [[15875, 16003]]}},
  {'head': 'Alberta',
   'type': 'country',
   'tail': 'Canada',
   'meta': {'spans': [[16002, 16130]]}},
  {'head': 'Canada',
   'type': 'contains administrative territorial entity',
   'tail': 'Alberta',
   'meta': {'spans': [[16002, 16130]]}},
  {'head': 'Hypertension',
   'type': 'has cause',
   'tail': 'Gestational diabetes',
   'meta': {'spans': [[16129, 16257]]}},
    {'head': 'hypervolemia', 'type': 'subclass of', 'tail': 'cardiovascular', 'meta': {'spans': [[0, 128]]}},
  {'head': 'diastolic dysfunction', 'type': 'subclass of', 'tail': 'cardiovascular', 'meta': {'spans': [[0, 128]]}},
  {'head': 'hypervolemia', 'type': 'has effect', 'tail': 'cardiovascular', 'meta': {'spans': [[0, 128]]}},
  {'head': 'systolic', 'type': 'part of', 'tail': 'arterial structure', 'meta': {'spans': [[125, 253]]}},
  {'head': 'diastolic', 'type': 'part of', 'tail': 'arterial structure', 'meta': {'spans': [[125, 253]]}},
  {'head': 'arterial structure', 'type': 'part of', 'tail': 'vascular system', 'meta': {'spans': [[125, 253]]}},
  {'head': 'vascular system', 'type': 'has part', 'tail': 'arterial structure', 'meta': {'spans': [[125, 253]]}},
  {'head': 'cohort study', 'type': 'subclass of', 'tail': 'retrospective', 'meta': {'spans': [[250, 378]]}},
  {'head': 'first singleton pregnancy', 'type': 'followed by', 'tail': 'second singleton pregnancy', 'meta': {'spans': [[250, 378]]}},
  {'head': 'first-trimester bleeding', 'type': 'part of', 'tail': 'pregnancy management', 'meta': {'spans': [[250, 378]]}},
  {'head': 'First-trimester bleeding', 'type': 'subclass of', 'tail': 'vaginal bleeding', 'meta': {'spans': [[375, 503]]}},
  {'head': 'First-trimester bleeding', 'type': 'instance of', 'tail': 'vaginal bleeding', 'meta': {'spans': [[375, 503]]}},
  {'head': 'first pregnancy', 'type': 'followed by', 'tail': 'second pregnancy', 'meta': {'spans': [[500, 628]]}},
  {'head': 'second pregnancy', 'type': 'follows', 'tail': 'first pregnancy', 'meta': {'spans': [[500, 628]]}},
  {'head': 'First-trimester bleeding', 'type': 'followed by', 'tail': 'second pregnancy', 'meta': {'spans': [[500, 628]]}},
  {'head': 'second pregnancy', 'type': 'follows', 'tail': 'First-trimester bleeding', 'meta': {'spans': [[500, 628]]}},
  {'head': 'Miscarriage', 'type': 'subclass of', 'tail': 'complication during early pregnancy', 'meta': {'spans': [[625, 753]]}},
  {'head': 'first-trimester bleeding', 'type': 'subclass of', 'tail': 'Miscarriage', 'meta': {'spans': [[625, 753]]}},
  {'head': 'Miscarriage', 'type': 'subclass of', 'tail': 'complication', 'meta': {'spans': [[625, 753]]}},
  {'head': 'meta-analysis', 'type': 'facet of', 'tail': 'systematic review', 'meta': {'spans': [[750, 878], [875, 1003]]}},
  {'head': 'systematic review', 'type': 'uses', 'tail': 'scientific evidence', 'meta': {'spans': [[750, 878]]}},
  {'head': 'meta-analysis', 'type': 'facet of', 'tail': 'scientific evidence', 'meta': {'spans': [[750, 878]]}},
  {'head': 'meta-analysis', 'type': 'subclass of', 'tail': 'systematic review', 'meta': {'spans': [[875, 1003]]}},
  {'head': 'meta-analysis', 'type': 'part of', 'tail': 'systematic review', 'meta': {'spans': [[875, 1003]]}},
  {'head': 'diastolic', 'type': 'opposite of', 'tail': 'systolic', 'meta': {'spans': [[1000, 1128]]}},
  {'head': 'diastolic', 'type': 'subclass of', 'tail': 'heart rate', 'meta': {'spans': [[1000, 1128], [1125, 1253]]}},
  {'head': 'systolic', 'type': 'opposite of', 'tail': 'diastolic', 'meta': {'spans': [[1000, 1128]]}},
  {'head': 'heart rate', 'type': 'subclass of', 'tail': 'beats', 'meta': {'spans': [[1125, 1253]]}},
  {'head': 'heart rate', 'type': 'has part', 'tail': 'beats/min', 'meta': {'spans': [[1125, 1253]]}},
  {'head': 'manual measurement', 'type': 'opposite of', 'tail': 'automated measurement', 'meta': {'spans': [[1250, 1378]]}},
  {'head': 'automated measurement', 'type': 'opposite of', 'tail': 'manual measurement', 'meta': {'spans': [[1250, 1378]]}},
  {'head': 'manual measurement', 'type': 'different from', 'tail': 'automated measurement', 'meta': {'spans': [[1250, 1378]]}},
  {'head': 'automated measurement', 'type': 'different from', 'tail': 'manual measurement', 'meta': {'spans': [[1250, 1378]]}},
  {'head': 'Manual', 'type': 'different from', 'tail': 'automated blood pressure measurement', 'meta': {'spans': [[1375, 1503]]}},
  {'head': 'automated blood pressure measurement', 'type': 'different from', 'tail': 'Manual', 'meta': {'spans': [[1375, 1503]]}},
  {'head': 'diastolic blood pressure', 'type': 'subclass of', 'tail': 'gestational blood pressure', 'meta': {'spans': [[1375, 1503]]}},
  {'head': 'Healthcare Cost and Utilization Project', 'type': 'country', 'tail': 'United States', 'meta': {'spans': [[1500, 1628]]}},
  {'head': 'Nationwide Inpatient Sample', 'type': 'part of', 'tail': 'Healthcare Cost and Utilization Project', 'meta': {'spans': [[1500, 1628]]}},
  {'head': 'Healthcare Cost and Utilization Project', 'type': 'has part', 'tail': 'Nationwide Inpatient Sample', 'meta': {'spans': [[1500, 1628]]}},
  {'head': 'superimposed preeclampsia', 'type': 'subclass of', 'tail': 'hypertension', 'meta': {'spans': [[1625, 1753]]}},
  {'head': 'preterm delivery', 'type': 'has cause', 'tail': 'hypertension', 'meta': {'spans': [[1625, 1753]]}},
  {'head': 'hypertension', 'type': 'has effect', 'tail': 'preterm delivery', 'meta': {'spans': [[1625, 1753]]}},
  {'head': 'postpartum period', 'type': 'follows', 'tail': 'pregnancy', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'MEDLINE', 'type': 'instance of', 'tail': 'electronic database', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'EMBASE', 'type': 'instance of', 'tail': 'electronic database', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'PsycInfo', 'type': 'instance of', 'tail': 'electronic database', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'World Trade Center disaster', 'type': 'point in time', 'tail': 'September 11th', 'meta': {'spans': [[1875, 2003]]}},
  {'head': 'earthquake', 'type': 'subclass of', 'tail': 'natural disaster', 'meta': {'spans': [[1875, 2003]]}},
  {'head': 'World Trade Center disaster', 'type': 'instance of', 'tail': 'terrorist attacks', 'meta': {'spans': [[1875, 2003]]}},
  {'head': 'postpartum', 'type': 'follows', 'tail': 'pregnant', 'meta': {'spans': [[2000, 2128]]}},
  {'head': 'postpartum', 'type': 'follows', 'tail': 'pregnancy', 'meta': {'spans': [[2000, 2128]]}},
  {'head': 'postpartum', 'type': 'facet of', 'tail': 'maternal mental health', 'meta': {'spans': [[2000, 2128]]}},
  {'head': 'pregnancy', 'type': 'has effect', 'tail': 'perinatal period', 'meta': {'spans': [[2125, 2253]]}},
  {'head': 'perinatal period', 'type': 'has cause', 'tail': 'pregnancy', 'meta': {'spans': [[2125, 2253]]}},
  {'head': 'pregnancy', 'type': 'followed by', 'tail': 'perinatal period', 'meta': {'spans': [[2125, 2253]]}},
  {'head': 'perinatal period', 'type': 'followed by', 'tail': 'postnatal year', 'meta': {'spans': [[2250, 2378]]}},
  {'head': 'postnatal year', 'type': 'follows', 'tail': 'perinatal period', 'meta': {'spans': [[2250, 2378]]}},
  {'head': 'postpartum', 'type': 'followed by', 'tail': 'postnatal year', 'meta': {'spans': [[2250, 2378]]}},
  {'head': 'maternal', 'type': 'child', 'tail': 'child', 'meta': {'spans': [[2375, 2503], [2625, 2753]]}},
  {'head': 'maternal mental disorder', 'type': 'subclass of', 'tail': 'mental health', 'meta': {'spans': [[2375, 2503]]}},
  {'head': 'maternal', 'type': 'has part', 'tail': 'child', 'meta': {'spans': [[2375, 2503]]}},
  {'head': 'mother', 'type': 'child', 'tail': 'infant', 'meta': {'spans': [[2500, 2628]]}},
  {'head': 'infant', 'type': 'mother', 'tail': 'mother', 'meta': {'spans': [[2500, 2628]]}},
  {'head': 'child', 'type': 'mother', 'tail': 'mother', 'meta': {'spans': [[2500, 2628]]}},
  {'head': 'perinatal self-harm', 'type': 'subclass of', 'tail': 'self-harm', 'meta': {'spans': [[2625, 2753]]}},
  {'head': 'pregnancy', 'type': 'has effect', 'tail': 'child', 'meta': {'spans': [[2625, 2753]]}},
  {'head': 'pregnancy', 'type': 'has effect', 'tail': 'delivery', 'meta': {'spans': [[2750, 2878]]}},
  {'head': 'delivery', 'type': 'has cause', 'tail': 'pregnancy', 'meta': {'spans': [[2750, 2878]]}},
  {'head': 'pregnancy', 'type': 'has effect', 'tail': 'labour', 'meta': {'spans': [[2750, 2878]]}},
  {'head': 'labour', 'type': 'has cause', 'tail': 'pregnancy', 'meta': {'spans': [[2750, 2878]]}},
  {'head': 'sex ratio', 'type': 'facet of', 'tail': 'multiple births', 'meta': {'spans': [[2875, 3003]]}},
  {'head': 'Prochownick', 'type': 'field of work', 'tail': 'multiple births', 'meta': {'spans': [[2875, 3003]]}},
  {'head': 'sex ratio', 'type': 'facet of', 'tail': 'multiple birth', 'meta': {'spans': [[2875, 3003]]}},
  {'head': 'cardiovascular dynamics', 'type': 'subclass of', 'tail': 'physiology', 'meta': {'spans': [[3000, 3128]]}},
  {'head': 'cardiovascular dynamics', 'type': 'part of', 'tail': 'physiology', 'meta': {'spans': [[3000, 3128]]}},
  {'head': 'renal function', 'type': 'part of', 'tail': 'physiology', 'meta': {'spans': [[3000, 3128]]}},
  {'head': 'pregnanediol', 'type': 'instance of', 'tail': 'sex steroid', 'meta': {'spans': [[3125, 3253]]}},
  {'head': 'pregnanediol', 'type': 'instance of', 'tail': 'pregnesterone', 'meta': {'spans': [[3125, 3253]]}},
  {'head': 'relaxin', 'type': 'instance of', 'tail': 'corticosteroid', 'meta': {'spans': [[3125, 3253]]}},
  {'head': 'pregnancy', 'type': 'has part', 'tail': 'mother', 'meta': {'spans': [[3250, 3378]]}},
  {'head': 'mother', 'type': 'part of', 'tail': 'pregnancy', 'meta': {'spans': [[3250, 3378]]}},
  {'head': 'pregnancy', 'type': 'has part', 'tail': 'fetus', 'meta': {'spans': [[3250, 3378]]}},
  {'head': 'pregnancy', 'type': 'has part', 'tail': 'foetus', 'meta': {'spans': [[3375, 3503]]}},
  {'head': 'pregnancy', 'type': 'has part', 'tail': 'placenta', 'meta': {'spans': [[3375, 3503]]}},
  {'head': 'conception', 'type': 'has part', 'tail': 'foetus', 'meta': {'spans': [[3375, 3503]]}},
  {'head': 'foetus', 'type': 'part of', 'tail': 'conception', 'meta': {'spans': [[3375, 3503]]}},
  {'head': 'albumin', 'type': 'part of', 'tail': 'plasma', 'meta': {'spans': [[3500, 3628]]}},
  {'head': 'first trimester', 'type': 'facet of', 'tail': 'fetus', 'meta': {'spans': [[3500, 3628]]}},
  {'head': 'first trimester', 'type': 'part of', 'tail': 'fetus', 'meta': {'spans': [[3500, 3628]]}},
  {'head': 'MEDLINE', 'type': 'instance of', 'tail': 'databases', 'meta': {'spans': [[3625, 3753]]}},
  {'head': 'MEDLINE', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[3625, 3753]]}},
  {'head': 'MEDLINE', 'type': 'instance of', 'tail': 'database', 'meta': {'spans': [[3625, 3753]]}},
  {'head': 'International Bibliography of the Social Sciences', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[3750, 3878]]}},
  {'head': 'CINAHL', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[3750, 3878]]}},
  {'head': 'International Bibliography of the Social Sciences', 'type': 'has part', 'tail': 'PubMed', 'meta': {'spans': [[3750, 3878]]}},
  {'head': 'final meta-analysis', 'type': 'instance of', 'tail': 'meta-analysis', 'meta': {'spans': [[3875, 4003]]}},
  {'head': 'odds ratio', 'type': 'facet of', 'tail': 'model', 'meta': {'spans': [[3875, 4003]]}},
  {'head': 'pooled odds ratio', 'type': 'subclass of', 'tail': 'odds ratio', 'meta': {'spans': [[3875, 4003]]}},
  {'head': 'Medical Birth Registry of Norway', 'type': 'country', 'tail': 'Norway', 'meta': {'spans': [[4000, 4128]]}},
  {'head': 'Medical Birth Registry', 'type': 'country', 'tail': 'Norway', 'meta': {'spans': [[4000, 4128]]}},
  {'head': 'The Medical Birth Registry of Norway', 'type': 'country', 'tail': 'Norway', 'meta': {'spans': [[4000, 4128]]}},
  {'head': 'postpartum bleeding', 'type': 'subclass of', 'tail': 'excess postpartum bleeding', 'meta': {'spans': [[4125, 4253]]}},
  {'head': 'postpartum bleeding', 'type': 'has effect', 'tail': 'excess postpartum bleeding', 'meta': {'spans': [[4125, 4253]]}},
  {'head': 'postpartum bleeding', 'type': 'subclass of', 'tail': 'postpartum bleeding', 'meta': {'spans': [[4125, 4253]]}},
  {'head': 'Cesarean section', 'type': 'subclass of', 'tail': 'delivery', 'meta': {'spans': [[4250, 4378]]}},
  {'head': 'Cesarean section', 'type': 'instance of', 'tail': 'type of delivery', 'meta': {'spans': [[4250, 4378]]}},
  {'head': 'first time mothers', 'type': 'different from', 'tail': 'more than one birth', 'meta': {'spans': [[4250, 4378]]}},
  {'head': 'more than one birth', 'type': 'different from', 'tail': 'first time mothers', 'meta': {'spans': [[4250, 4378]]}}
]

# Extract nodes (head and tail) and remove duplicates using set
nodes = set()
for item in data:
    nodes.add(item['head'])
    nodes.add(item['tail'])

# Print the unique nodes
print(nodes)


{'second singleton pregnancy', 'systematic review', 'Anti-gravity', 'first pregnancy', 'Manual', 'Washington, D.C.', 'Iran', 'First-trimester bleeding', 'Tehran', 'preterm delivery', 'hypervolemia', 'Growth factor', 'beats/min', 'Pregnancy', '1991', 'Heat', 'EMBASE', 'Hamilton, Ontario', 'first-trimester bleeding', 'database', 'pregnancy', 'labour', 'diastolic blood pressure', 'Gravity', 'Diabetes and pregnancy', 'postnatal year', 'CINAHL', 'Infant', 'manual measurement', 'Apgar score', 'vaginal bleeding', 'Scientific journal', 'University of Toronto', 'complication during early pregnancy', 'infant', 'self-harm', 'Physiology', 'Confidence interval', 'placenta', 'diastolic dysfunction', 'Insulin resistance', 'Systematic review', 'Jogging', 'Nationwide Inpatient Sample', 'Glycemic', 'cardiovascular', 'Caesarean section', 'Birth weight', 'Low birth weight', 'cohort study', 'albumin', 'first trimester', 'Neural tube defect', 'Randomized controlled trial', 'Exercise', 'MEDLINE', 'arterial s

In [ ]:
nodes

{'1985',
 '1991',
 '2',
 '2014',
 '2024',
 'Aerobic exercise',
 'Aerobics',
 'Air conditioning',
 'Alberta',
 'American College of Obstetricians and Gynecologists',
 'American Journal of Obstetrics and Gynecology',
 'Anatomy',
 'Anti-gravity',
 'Apgar',
 'Apgar score',
 'Birth defect',
 'Birth weight',
 'Blood pressure',
 'Blood sugar level',
 'British Journal of Sports Medicine',
 'CINAHL',
 'Caesarean section',
 'Canada',
 'Cardiology',
 'Cesarean section',
 'Childbirth',
 'Chlorine',
 'Clinical trial',
 'Cochrane Library',
 'Confidence interval',
 'Diabetes',
 'Diabetes and pregnancy',
 'Diagnosis',
 'EMBASE',
 'Elsevier',
 'Epidemiology',
 'Exercise',
 'First-trimester bleeding',
 'Geneva',
 'Gestational diabetes',
 'Gestational hypertension',
 'Glucose tolerance test',
 'Glycemic',
 'Gravity',
 'Growth factor',
 'Growth hormone 2',
 'Hamilton, Ontario',
 'Healthcare Cost and Utilization Project',
 'Heat',
 'Hypertension',
 'Hypoglycemia',
 'Icahn School of Medicine at Mount Sinai'

In [ ]:
def segment_nodes(csv_file_path):
  """
  This function segments nodes from a CSV file into original and duplicate lists.

  Args:
      csv_file_path: Path to the CSV file containing node information.

  Returns:
      A tuple containing two lists: original_nodes and duplicate_nodes.
  """
  original_nodes = []
  duplicate_nodes = []

  with open(csv_file_path, "r") as csvfile:
    next(csvfile)  # Skip the header row
    for line in csvfile:
      node_info = line.strip().split("\t")
      print(node_info)
      node_name, entity_type, _ = node_info

      # Check for duplicates using the node name
      if 'Duplicate' in entity_type:
        duplicate_nodes.append(node_info)
      else:
        original_nodes.append(node_info)

  return original_nodes, duplicate_nodes

# Example usage
csv_file_path = "nodes.tsv"  # Replace with your actual CSV filepath

original_nodes, duplicate_nodes = segment_nodes(csv_file_path)

print("Original nodes:")
for node in original_nodes:
  print(f"\t- {node}")

print("\nDuplicate nodes:")
for node in duplicate_nodes:
  print(f"\t- {node}")


['1985', 'Event', 'Represents a specific year, possibly relevant to historical events.']
['1991', 'Event', 'Represents a specific year, possibly relevant to historical events.']
['2', 'Quantity', 'Represents a numerical value, likely indicating an area unit in this context.']
['2014', 'Event', 'Represents a specific year, possibly relevant to historical events.']
['2024', 'Event', 'Represents the current year, likely not relevant as an entity.']
['Aerobic exercise', 'Concept', 'Represents a type of physical activity.']
['Aerobics', 'Concept', 'Represents a type of exercise program.']
['Air conditioning', 'Concept', 'Represents a technology for climate control.']
['Alberta', 'Location', 'Represents a Canadian province.']
['American College of Obstetricians and Gynecologists', 'Organization', 'Represents a professional medical organization.']
['American Journal of Obstetrics and Gynecology', 'Scientific journal', 'Represents a specialized medical publication.']
['Anatomy', 'Concept', 'Re

In [ ]:

def create_entity(tx, node_name, entity_type,summary):
    tx.run("MERGE (n:(%s) {name: '%s',summary:'%s'})  RETURN n;"%(entity_type,node_name,summary))
          #  node_name=node_name, entity_type=entity_type,summary=summary)

with driver.session() as session:
    with open("output.tsv", "r") as file:
        next(file)  # Skip the header row
        for line in file:
            node_name, entity_type, summary = line.strip().split("\t")
            if ' ' in entity_type:entity_type=entity_type.replace(' ', '_')
            if "'s" in summary:summary=summary.replace("'s","")
            if node_name == "labour":continue
            session.write_transaction(create_entity, node_name, entity_type,summary)

driver.close()

<ipython-input-20-b797f13bcd0a>:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_entity, node_name, entity_type,summary)


In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("neo4j+s://349ebe81.databases.neo4j.io", auth=("neo4j", "Uubn7xyMygLbDyAHQfPtt06qtSF1elaXDwEqUvLM_WA"))


In [ ]:
 "MERGE (:%s {name: '%s'})

In [ ]:
import re
def extract_names_and_entity_types(file_path):
    entity_dict = {}

    # Open the text file for reading
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize variables to store the current name and entity type
    current_name = None
    current_entity_type = None

    # Iterate through the lines in the file
    for line in lines:
        if re.match(r"^\d+\.\s+'(.*?)'", line):
            current_name = re.search(r"'(.*?)'", line).group(1)
        elif "Entity Type:" in line:
            current_entity_type = line.strip().split("Entity Type:")[1].strip()
        if current_name and current_entity_type:
            entity_dict[current_name] = current_entity_type
            current_name = None
            current_entity_type = None

    return entity_dict

# Example usage:
file_path = 'message.txt'  # Replace with the path to your input file
result = extract_names_and_entity_types(file_path)

# Print the extracted names and entity types
for name, entity_type in result.items():
    print(f"Name: {name}, Entity Type: {entity_type}")


Name: Hypervolemia, Entity Type: Medical Condition
Name: Circulatory System, Entity Type: Biological System
Name: Heart Failure with Preserved Ejection Fraction, Entity Type: Medical Condition
Name: Cohort Study, Entity Type: Research Study Type
Name: Retrospective, Entity Type: Research Study Type
Name: Meta-analysis, Entity Type: Research Study Type
Name: Systematic Review, Entity Type: Research Study Type
Name: Scientific Evidence, Entity Type: Concept
Name: Diastole, Entity Type: Cardiac Cycle Phase
Name: Systole, Entity Type: Cardiac Cycle Phase
Name: Heart Rate, Entity Type: Physiological Measurement
Name: Healthcare Cost and Utilization Project, Entity Type: Healthcare Database/Project
Name: United States, Entity Type: Geographic Location
Name: Preterm Birth, Entity Type: Medical Condition
Name: Hypertension, Entity Type: Medical Condition
Name: Postpartum Period, Entity Type: Medical Time Period
Name: Pregnancy, Entity Type: Medical Condition
Name: MEDLINE, Entity Type: Bibliog

In [ ]:
def evaluate_link_coverage(relation_triplets):
  """Evaluates the link coverage of a knowledge graph.

  """

  # Create a set of all possible node pairs.
  all_node_pairs = set()
  for subject_node, relation, object_node in relation_triplets:
    all_node_pairs.add((subject_node, object_node))

  # Count the number of node pairs that are linked in the knowledge graph.
  linked_node_pairs = 0
  for subject_node, relation, object_node in relation_triplets:
    linked_node_pairs += 1

  # Calculate the link coverage.
  link_coverage = linked_node_pairs / len(all_node_pairs)

  return link_coverage

In [ ]:
evaluate_link_coverage(relation_triplets)

1.2105263157894737

In [ ]:
def convert_to_relation_triplets(data):

  relation_triplets = []
  for item in data:
    head = item["head"]
    relation = item["type"]
    tail = item["tail"]
    relation_triplets.append((head, relation, tail))

  return relation_triplets



data = [
  {"head": "Hypervolemia", "type": "subclass of", "tail": "Circulatory system", "meta": {"spans": [[0, 128]]}},
  {"head": "Heart failure with preserved ejection fraction", "type": "subclass of", "tail": "Circulatory system", "meta": {"spans": [[0, 128]]}},
  {'head': 'Hypervolemia', 'type': 'has effect', 'tail': 'Circulatory system', 'meta': {'spans': [[0, 128]]}},
  {'head': 'Cohort study', 'type': 'subclass of', 'tail': 'Retrospective', 'meta': {'spans': [[250, 378]]}},
  {'head': 'Meta-analysis', 'type': 'facet of', 'tail': 'Systematic review', 'meta': {'spans': [[750, 878], [875, 1003]]}},
  {'head': 'Systematic review', 'type': 'uses', 'tail': 'Scientific evidence', 'meta': {'spans': [[750, 878]]}},
  {'head': 'Meta-analysis', 'type': 'facet of', 'tail': 'Scientific evidence', 'meta': {'spans': [[750, 878]]}},
  {'head': 'Meta-analysis', 'type': 'subclass of', 'tail': 'Systematic review', 'meta': {'spans': [[875, 1003]]}},
  {'head': 'Meta-analysis', 'type': 'part of', 'tail': 'Systematic review', 'meta': {'spans': [[875, 1003]]}},
  {'head': 'Diastole', 'type': 'opposite of', 'tail': 'Systole', 'meta': {'spans': [[1000, 1128]]}},
  {'head': 'Diastole', 'type': 'subclass of', 'tail': 'Heart rate', 'meta': {'spans': [[1000, 1128], [1125, 1253]]}},
  {'head': 'Systole', 'type': 'opposite of', 'tail': 'Diastole', 'meta': {'spans': [[1000, 1128]]}},
  {'head': 'Healthcare Cost and Utilization Project', 'type': 'country', 'tail': 'United States', 'meta': {'spans': [[1500, 1628]]}},
  {'head': 'Preterm birth', 'type': 'has cause', 'tail': 'Hypertension', 'meta': {'spans': [[1625, 1753]]}},
  {'head': 'Hypertension', 'type': 'has effect', 'tail': 'Preterm birth', 'meta': {'spans': [[1625, 1753]]}},
  {'head': 'Postpartum period', 'type': 'follows', 'tail': 'Pregnancy', 'meta': {'spans': [[1750, 1878], [2000, 2128]]}},
  {'head': 'MEDLINE', 'type': 'instance of', 'tail': 'Bibliographic database', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'Embase', 'type': 'instance of', 'tail': 'Bibliographic database', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'PsycINFO', 'type': 'instance of', 'tail': 'Bibliographic database', 'meta': {'spans': [[1750, 1878]]}},
  {'head': 'Earthquake', 'type': 'subclass of', 'tail': 'Natural disaster', 'meta': {'spans': [[1875, 2003]]}},
  {'head': 'Pregnancy', 'type': 'has effect', 'tail': 'Prenatal development', 'meta': {'spans': [[2125, 2253]]}},
  {'head': 'Prenatal development', 'type': 'has cause', 'tail': 'Pregnancy', 'meta': {'spans': [[2125, 2253]]}},
  {'head': 'Pregnancy', 'type': 'followed by', 'tail': 'Prenatal development', 'meta': {'spans': [[2125, 2253]]}},
  {'head': 'Mother', 'type': 'child', 'tail': 'Child', 'meta': {'spans': [[2375, 2503], [2625, 2753]]}},
  {'head': 'Mother', 'type': 'has part', 'tail': 'Child', 'meta': {'spans': [[2375, 2503]]}},
  {'head': 'Mother', 'type': 'child', 'tail': 'Infant', 'meta': {'spans': [[2500, 2628]]}},
  {'head': 'Infant', 'type': 'mother', 'tail': 'Mother', 'meta': {'spans': [[2500, 2628]]}},
  {'head': 'Child', 'type': 'mother', 'tail': 'Mother', 'meta': {'spans': [[2500, 2628]]}},
  {'head': 'Pregnancy', 'type': 'has effect', 'tail': 'Child', 'meta': {'spans': [[2625, 2753]]}},
  {'head': 'Sex ratio', 'type': 'facet of', 'tail': 'Multiple birth', 'meta': {'spans': [[2875, 3003]]}},
  {'head': 'Assessment of kidney function', 'type': 'part of', 'tail': 'Physiology', 'meta': {'spans': [[3000, 3128]]}},
  {'head': 'Pregnanediol', 'type': 'instance of', 'tail': 'Sex hormone', 'meta': {'spans': [[3125, 3253]]}},
  {'head': 'Relaxin', 'type': 'instance of', 'tail': 'Corticosteroid', 'meta': {'spans': [[3125, 3253]]}},
  {'head': 'Pregnancy', 'type': 'has part', 'tail': 'Mother', 'meta': {'spans': [[3250, 3378]]}},
  {'head': 'Mother', 'type': 'part of', 'tail': 'Pregnancy', 'meta': {'spans': [[3250, 3378]]}},
  {'head': 'Pregnancy', 'type': 'has part', 'tail': 'Fetus', 'meta': {'spans': [[3250, 3378], [3375, 3503]]}},
  {'head': 'Pregnancy', 'type': 'has part', 'tail': 'Placenta', 'meta': {'spans': [[3375, 3503]]}},
  {'head': 'Pregnancy', 'type': 'facet of', 'tail': 'Fetus', 'meta': {'spans': [[3500, 3628]]}},
  {'head': 'Pregnancy', 'type': 'part of', 'tail': 'Fetus', 'meta': {'spans': [[3500, 3628]]}},
  {'head': 'MEDLINE', 'type': 'instance of', 'tail': 'Database', 'meta': {'spans': [[3625, 3753]]}},
  {'head': 'MEDLINE', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[3625, 3753]]}},
  {'head': 'International Bibliography of the Social Sciences', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[3750, 3878]]}},
  {'head': 'CINAHL', 'type': 'part of', 'tail': 'PubMed', 'meta': {'spans': [[3750, 3878]]}},
  {'head': 'International Bibliography of the Social Sciences', 'type': 'has part', 'tail': 'PubMed', 'meta': {'spans': [[3750, 3878]]}},
  {'head': 'Odds ratio', 'type': 'facet of', 'tail': 'Model', 'meta': {'spans': [[3875, 4003]]}},
  {'head': 'Postpartum bleeding', 'type': 'subclass of', 'tail': 'Postpartum bleeding', 'meta': {'spans': [[4125, 4253]]}}
]

relation_triplets = convert_to_relation_triplets(data)

print(relation_triplets)


[('Hypervolemia', 'subclass of', 'Circulatory system'), ('Heart failure with preserved ejection fraction', 'subclass of', 'Circulatory system'), ('Hypervolemia', 'has effect', 'Circulatory system'), ('Cohort study', 'subclass of', 'Retrospective'), ('Meta-analysis', 'facet of', 'Systematic review'), ('Systematic review', 'uses', 'Scientific evidence'), ('Meta-analysis', 'facet of', 'Scientific evidence'), ('Meta-analysis', 'subclass of', 'Systematic review'), ('Meta-analysis', 'part of', 'Systematic review'), ('Diastole', 'opposite of', 'Systole'), ('Diastole', 'subclass of', 'Heart rate'), ('Systole', 'opposite of', 'Diastole'), ('Healthcare Cost and Utilization Project', 'country', 'United States'), ('Preterm birth', 'has cause', 'Hypertension'), ('Hypertension', 'has effect', 'Preterm birth'), ('Postpartum period', 'follows', 'Pregnancy'), ('MEDLINE', 'instance of', 'Bibliographic database'), ('Embase', 'instance of', 'Bibliographic database'), ('PsycINFO', 'instance of', 'Bibliogra

In [ ]:
triples = []

for item in kdict['relations']:

    # Extract components of each triple from the dictionary
    head = item['head']  # Subject
    relation = item['type']  # Predicate (relationship)
    tail = item['tail']  # Object

    # Create a tuple representing the triple and add it to the 'triples' list
    triples.append((head, relation, tail))

# Print the list of triples
print(triples)


NameError: name 'kdict' is not defined

In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.18.0-py3-none-any.whl size=273862 sha256=453893b4fd80e6ad367b1aa08aa3f45a99657edd2ba5c24dabcb0a7ddfffeb02
  Stored in directory: /root/.cache/pip/wheels/e7/e1/a0/dd7c19192f5383ff57d02a6c126cbfe4b7b2ae82f70c6994ce
Successfully built neo4j


In [ ]:
import json
import logging
from neo4j import GraphDatabase
from neo4j.exceptions import Neo4jError



if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    # uri = "neo4j+s://e38739c4.databases.neo4j.io"

    uri = "neo4j+s://11a69b53.databases.neo4j.io"
    user = "neo4j"
    password = "hzAg1YwoNuo9-xcIljhBBQsV8D5_RPkv7gMjC5eyWsk"

    # app = App(uri, user, password)
    AUTH = (user,password)
    with GraphDatabase.driver(uri, auth=AUTH) as driver:
      driver.verify_connectivity()

      for name, entity_type in result.items():
        print(f"Name: {name}, Entity Type: {entity_type}")
        # name=name.replace(' ', '_')
        entity_type=entity_type.replace(' ', '_')
        parts = entity_type.split('/')
        if len(parts) > 1:
          parts[-2] = ''
        entity_type = ''.join(parts)

        driver.execute_query( "MERGE (:%s {name: '%s'}) "%(entity_type,name),
        database_="neo4j",
        )

      for n1,r,n2 in relation_triplets:
        # n1=n1.replace(' ', '_')
        r=r.replace(' ', '_')
        # n2=n2.replace(' ', '_')
        print(n1,r,n2)
        query = '''MATCH (a),(b)
          WHERE a.name = '%s' and b.name = '%s'
          MERGE (a)-[rel:%s]->(b)
          RETURN a.name, rel, b.name
          '''%(n1,n2,r)
        print(driver.execute_query(query,database="neo4j"))




Name: Hypervolemia, Entity Type: Medical Condition
Name: Circulatory System, Entity Type: Biological System
Name: Heart Failure with Preserved Ejection Fraction, Entity Type: Medical Condition
Name: Cohort Study, Entity Type: Research Study Type
Name: Retrospective, Entity Type: Research Study Type
Name: Meta-analysis, Entity Type: Research Study Type
Name: Systematic Review, Entity Type: Research Study Type
Name: Scientific Evidence, Entity Type: Concept
Name: Diastole, Entity Type: Cardiac Cycle Phase
Name: Systole, Entity Type: Cardiac Cycle Phase
Name: Heart Rate, Entity Type: Physiological Measurement
Name: Healthcare Cost and Utilization Project, Entity Type: Healthcare Database/Project
Name: United States, Entity Type: Geographic Location
Name: Preterm Birth, Entity Type: Medical Condition
Name: Hypertension, Entity Type: Medical Condition
Name: Postpartum Period, Entity Type: Medical Time Period
Name: Pregnancy, Entity Type: Medical Condition
Name: MEDLINE, Entity Type: Bibliog

In [ ]:
for i ,j,k in relation_triplets:
  print(i,j,k)

Hypervolemia subclass of Circulatory system
Heart failure with preserved ejection fraction subclass of Circulatory system
Hypervolemia has effect Circulatory system
Cohort study subclass of Retrospective
Meta-analysis facet of Systematic review
Systematic review uses Scientific evidence
Meta-analysis facet of Scientific evidence
Meta-analysis subclass of Systematic review
Meta-analysis part of Systematic review
Diastole opposite of Systole
Diastole subclass of Heart rate
Systole opposite of Diastole
Healthcare Cost and Utilization Project country United States
Preterm birth has cause Hypertension
Hypertension has effect Preterm birth
Postpartum period follows Pregnancy
MEDLINE instance of Bibliographic database
Embase instance of Bibliographic database
PsycINFO instance of Bibliographic database
Earthquake subclass of Natural disaster
Pregnancy has effect Prenatal development
Prenatal development has cause Pregnancy
Pregnancy followed by Prenatal development
Mother child Child
Mother h